### Dataset Creation

This is the notebook to train the classification of Diabetic Retinography with CNNs

This notebook contains the following
1. Dataset Creation and Augmentation
2. Train and Eval Functions
3. CNN Class Models (InceptionV3, ResNet50, ResNet152, EfficientNet, DenseNet, VGG16, MaxViT)

This notebook assumes the following project structure:
```bash
Root
├── notebooks
│   └── notebook1.ipynb
└── input
    └── Data
        ├── DDR
        │   ├── Train
        │   └── Test
        ── BEN
        │   ├── Train
        │   └── Test
        ├── CLAHE
        │   ├── Train
        │   └── Test
        ├── UNET_Binary
        │   ├── Train
        │   └── Test
        └── UNET_Multiclass
            ├── Train
            └── Test
```

If you do not have the dataset, please download it from our Google Drive

In [ ]:
#Necessary Imports
import torch
from torchvision import datasets, transforms, models
from sklearn.svm import SVC
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc

### Dataset Creation

In [ ]:
# Parameter required
image_size = (299,299)
batch_size = 64

# Defining Train Transforms
train_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.RandomHorizontalFlip(p=0.5),  # Flip horizontally with a 50% probability
    transforms.RandomVerticalFlip(p=0.5),  # Flip vertically with a 50% probability
    transforms.RandomAffine(
        degrees=360,  # Rotation
        translate=(0.1, 0.1),  # Translation
        scale=(0.8, 1.2) #Zooming
    ),
    transforms.ToTensor()
])


# Defining Evaluation Transforms, no data augmentation
eval_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor()
])

#### Ben Graham dataset

In [ ]:
# Create PyTorch datasets for training and validation
ben_train_dataset = datasets.ImageFolder(
                    root='../input/Classification/BEN/train', 
                    transform=train_transform
                    )
ben_val_dataset = datasets.ImageFolder(
                    root='../input/Classification/BEN/val', 
                    transform=eval_transform
                    )
ben_test_dataset = datasets.ImageFolder(
                    root='../input/Classification/BEN/test', 
                    transform=eval_transform
                    )

# Create PyTorch dataloaders for training and validation
ben_train_dataloader = torch.utils.data.DataLoader(
                    ben_train_dataset,
                    batch_size=batch_size, 
                    shuffle=True
                    )
ben_val_dataloader = torch.utils.data.DataLoader(
                    ben_val_dataset, 
                    batch_size=batch_size, 
                    shuffle=True
                    )
ben_test_dataloader = torch.utils.data.DataLoader(
                    ben_test_dataset, 
                    batch_size=batch_size, 
                    shuffle=True
                    )

#### OPENCV dataset

In [ ]:
# Create PyTorch datasets for training and validation
opencv_train_dataset = datasets.ImageFolder(
                    root='../input/Classification/OPENCV/train', 
                    transform=train_transform
                    )
opencv_val_dataset = datasets.ImageFolder(
                    root='../input/Classification/OPENCV/val', 
                    transform=eval_transform
                    )
opencv_test_dataset = datasets.ImageFolder(
                    root='../input/Classification/OPENCV/test', 
                    transform=eval_transform
                    )

# Create PyTorch dataloaders for training and validation
opencv_train_dataloader = torch.utils.data.DataLoader(
                    opencv_train_dataset,
                    batch_size=batch_size, 
                    shuffle=True
                    )
opencv_val_dataloader = torch.utils.data.DataLoader(
                    opencv_val_dataset, 
                    batch_size=batch_size, 
                    shuffle=True
                    )
opencv_test_dataloader = torch.utils.data.DataLoader(
                    opencv_test_dataset, 
                    batch_size=batch_size, 
                    shuffle=True
                    )

#### UNET_Binary Dataset

In [ ]:
# Create PyTorch datasets for training and validation
unetb_train_dataset = datasets.ImageFolder(
                    root='../input/Classification/UNET_Binary/train', 
                    transform=train_transform
                    )
unetb_val_dataset = datasets.ImageFolder(
                    root='../input/Classification/UNET_Binary/val', 
                    transform=eval_transform
                    )
unetb_test_dataset = datasets.ImageFolder(
                    root='../input/Classification/UNET_Binary/test', 
                    transform=eval_transform
                    )

# Create PyTorch dataloaders for training and validation
unetb_train_dataloader = torch.utils.data.DataLoader(
                    unetb_train_dataset,
                    batch_size=batch_size, 
                    shuffle=True
                    )
unetb_val_dataloader = torch.utils.data.DataLoader(
                    unetb_val_dataset, 
                    batch_size=batch_size, 
                    shuffle=True
                    )
unetb_test_dataloader = torch.utils.data.DataLoader(
                    unetb_test_dataset, 
                    batch_size=batch_size, 
                    shuffle=True
                    )

#### UNET_Multiclass Dataset

In [ ]:
# Create PyTorch datasets for training and validation
unetm_train_dataset = datasets.ImageFolder(
                    root='../input/Classification/UNET_Multiclass/train', 
                    transform=train_transform
                    )
unetm_val_dataset = datasets.ImageFolder(
                    root='../input/Classification/UNET_Multiclass/val', 
                    transform=eval_transform
                    )
unetm_test_dataset = datasets.ImageFolder(
                    root='../input/Classification/UNET_Multiclass/test', 
                    transform=eval_transform
                    )

# Create PyTorch dataloaders for training and validation
unetm_train_dataloader = torch.utils.data.DataLoader(
                    unetm_train_dataset,
                    batch_size=batch_size, 
                    shuffle=True
                    )
unetm_val_dataloader = torch.utils.data.DataLoader(
                    unetm_val_dataset, 
                    batch_size=batch_size, 
                    shuffle=True
                    )
unetm_test_dataloader = torch.utils.data.DataLoader(
                    unetm_test_dataset, 
                    batch_size=batch_size, 
                    shuffle=True
                    )

#### Visualisation

In [ ]:
def visualise_img(dataloader, class_list: list):
    """
    Function to visualize the first 9 images of the dataset.

    Args:
        dataloader (DataLoader): PyTorch DataLoader object containing the dataset to visualize.
        class_list (list): List of class labels.
    """
    #Get the first batch of images and labels
    train_images, train_labels = next(iter(dataloader))
    batch_size = train_images.size(0)  # Get the batch size

    #Print the shape of the batch
    print(f"Images batch shape: {train_images.size()}")
    print(f"Labels batch shape: {train_labels.size()}")

    #Create a 3x3 grid for visualization
    fig, axes = plt.subplots(3, 3, figsize=(9, 9))

    for i in range(3):
        for j in range(3):
            #Get the index of the image in the batch
            index = i * 3 + j

            if index < batch_size:
                #Prepare image to print
                img = train_images[index].squeeze().numpy().transpose((1, 2, 0))
                label = train_labels[index].item()

                #Plot the image
                axes[i, j].imshow(img)
                axes[i, j].axis('off')
                axes[i, j].set_title(f'Label: {label}, {class_list[label]}', loc='left')

    plt.tight_layout()
    plt.show()

In [ ]:
visualise_img(ben_train_dataloader, ben_train_dataset.classes)

In [ ]:
visualise_img(opencv_train_dataloader, opencv_train_dataset.classes)

In [ ]:
visualise_img(unetb_train_dataloader, unetb_train_dataset.classes)

In [ ]:
visualise_img(unetm_train_dataloader, unetm_train_dataset.classes)

### Train, Eval Functions for CNN

In [ ]:
def calculate_dr_class_weights(dataset):
    """
    Function to calculate class weights
    Class Weights of i = Total Num of Samples / Total Num of samples of Class i * Num of classes

    We calculate the average of samples per class if it was equally distributed and then calculate the class weights based on the difference of actual vs ideal

    Args:
        dataset: train dataset

    Returns:
        weights (np.ndarray): array of size n_classes with the weights of each class in each index
    """
    # Counting the number of samples in each class
    class_counts = np.bincount(dataset.targets)
    total_samples = sum(class_counts)
    num_classes = len(class_counts)
    
    # Calculating class weights inversely proportional to the number of samples in each class
    weights = total_samples / (num_classes * class_counts)

    print("Class Weights:", weights)
    
    return weights

In [ ]:
def eval(model, 
         criterion, 
         img_size:tuple,
         val_dataloader, 
         device='cuda'):
    """
    Evaluation function for finetuning CNN models with a model object,
    incorporating average sensitivity for a multiclass problem.

    Sensitivity function: True Positives / (True Positives + False Negatives)

    Args:
        model: model to be trained
        criterion: loss function
        img_size (tuple): image size of dataset for model. All inputs will be resized to image size
        val_dataloader: val / test dataloader
        device (str, optional): 'cpu' or 'cuda', defaults to cuda.

    Returns:
        val_loss: float of the average val loss.
        val_accuracy: float of the accuracy.
        val_sensitivity: float of the average sensitivity across all classes.
    """

    model = model.to(device)
    
    #set model to eval mode
    model.eval()

    #variables 
    val_loss = 0.0
    correct = 0
    total = 0
    n_classes = 5
    true_positives = [0] * n_classes #stores num of true positives per class
    actual_positives = [0] * n_classes #stores total number of positives per class
    total_sensitivity = 0

    with torch.no_grad():
        for image, label in val_dataloader:
            batch_sensitivity = 0

            #resize image with bilinear, same as torchvision.transforms.Resize()
            image = torch.nn.functional.interpolate(image, size=img_size, mode='bilinear') 
            image, label = image.to(device), label.to(device)

            outputs = model(image)  #predict label
            loss = criterion(outputs, label)  #calculate loss
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1) #get prediction
            total += label.size(0)
            correct += (predicted == label).sum().item()

            for i in range(n_classes):
                true_positives[i] += ((predicted == i) & (label == i)).sum().item() #true positives
                actual_positives[i] += (label == i).sum().item() #true positives + false negatives
            
                if (label == i).sum().item() > 0:
                    batch_sensitivity += true_positives[i] / actual_positives[i]

                else:
                    pass
            
            total_sensitivity += batch_sensitivity/n_classes #average sensitivity for batch

    # Calculate accuracy, avg loss, and avg sensitivity
    accuracy = (correct / total) * 100
    avg_val_loss = val_loss / len(val_dataloader)
    avg_sensitivity = total_sensitivity/len(val_dataloader)
    
    return avg_val_loss, accuracy, avg_sensitivity


In [ ]:
def train(model, 
          criterion, 
          optimiser, 
          img_size:tuple,
          train_dataloader, 
          val_dataloader=None, 
          saving_metric:str='sensitivity',
          num_epochs:int=25, 
          device:str='cuda', 
          model_name:str=None):
    """
    Training Function to train model
    Runs validation for each epoch to calculate: Validation Loss, Validation Accuracy, Validation Sensitivity
    Best and last model will be saved to ../models/cnn under {model_name}_best.pt and {model_name}_last.pt
    
    Args:
        model: model to be trained
        criterion: loss function
        optimiser: optimiser chosen
        img_size (tuple): image size of dataset for model. All inputs will be resized to image size
        train_dataloader: train dataloader
        val_dataloader (optional): val dataloader, if None no validation will be calculated. Defaults to None.
        saving_metric (str, optional): saving metrics for best model, either "loss", "accuracy", or "sensitivity". Defaults to 'sensitivity'.
        num_epochs (int, optional): number of training epochs. Defaults to 25.
        device (str, optional): cuda or cpu. Defaults to 'cuda'.
        model_name (str, optional): model name to be saved, if None no model will be saved. Defaults to None.

    Returns:
        results_dataframe: dataframe of [model, train_loss, val_loss, val_accuracy, val_sensitivity] where each row is each epoch
    """

    if saving_metric not in ["loss", "accuracy", "sensitivity"]:
        raise Exception("Invalid saving metrics found, please only use loss, accuracy or sensitivity")

    #initialising results container
    results = pd.DataFrame(columns=["Model", "train loss", "val loss", "val accuracy", "val sensitivity"])
    
    #placeholders
    val_loss = ''
    val_accuracy = ''
    val_sensitivity = ''


    for epoch in range(num_epochs):

        #initialising training
        model.train()
        training_loss = 0.0

        for image, label in tqdm(train_dataloader):
            
            #resize image with bilinear, same as torchvision.transforms.Resize()
            image = torch.nn.functional.interpolate(image, size=img_size, mode='bilinear') 
            image, label = image.to(device), label.to(device)

            optimiser.zero_grad()
            outputs = model(image)
            #calculate loss and train model
            loss = criterion(outputs, label)
            loss.backward()
            optimiser.step()
            training_loss += loss.item() #update the training loss

        epoch_loss = training_loss / len(train_dataloader) #calculate training loss in epoch
        print(f"Epoch {epoch+1} completed, training loss: {epoch_loss}")

        #validation
        if val_dataloader is not None:
            model.eval()  #set model to evaluate mode
            val_loss, val_accuracy,  val_sensitivity = eval(model=model, 
                                                               criterion=criterion,
                                                                val_dataloader=val_dataloader, 
                                                                img_size = img_size,
                                                                device=device) 
            
            print(f"Validation loss: {val_loss}, Validation Accuracy: {val_accuracy:.2f}, Validation Sensitivty: {val_sensitivity:2f}")
            

            if saving_metric == 'loss' and len(results) > 0 and val_loss < min(results['val loss'].to_list()):
                torch.save(model, f'../models/cnn/{model_name}_best.pt')
                print("Best model saved")

            elif saving_metric == 'accuracy' and len(results) > 0 and val_accuracy > max(results['val accuracy'].to_list()):
                torch.save(model, f'../models/cnn/{model_name}_best.pt')
                print("Best model saved")

            elif saving_metric == 'sensitivity' and len(results) > 0 and val_sensitivity > max(results['val sensitivity'].to_list()):
                torch.save(model, f'../models/cnn/{model_name}_best.pt')
                print("Best model saved")
            
        #updating results
        results.loc[len(results)] = [model_name, epoch_loss, val_loss, val_accuracy, val_sensitivity]


    #save the last model
    if model_name is not None:
        torch.save(model, f'../models/cnn/{model_name}_last.pt')

    return results


### Train, Eval Functions for SVM

In [ ]:
def generate_model_outputs(model_list:list, dataloader, device:str='cuda'):
    """
    Function to generate the outputs of the model from dataloader and add them to a dictionary
    Expects model_list to be either [[]] or [[], []] 

    Args:
        model_list (list): nested list of either [[[model, model_name, img_size], [model, model_name, img_size]]] or [[[model, model_name, img_size], [model, model_name, img_size]], [[model, model_name, img_size], [model, model_name, img_size]]]
        dataloader: dataloader to run outputs for
        device (str, optional): cuda or cpu. Defaults to 'cuda'.

    Returns:
        model_outputs (dict): Dict in the format of {model_name: [output]}
        true_labels (list): list of labels for each output
    """
    model_outputs = {}
    true_labels = []

    with torch.no_grad():
        for images, label in tqdm(dataloader):
            #get model output and update in dictionary
            for model, model_name, img_size in model_list[0]:
                model.eval()
                images = torch.nn.functional.interpolate(images, img_size, mode='bilinear').to(device)
                output = model(images)
                existing_results = model_outputs.get(model_name, [])
                existing_results.append(output)
                model_outputs[model_name] = existing_results

                #for nested list, run for second batch of models
                if len(model_list) > 1:
                    for model, model_name, img_size in model_list[1]:
                        model.eval()
                        images = torch.nn.functional.interpolate(images, img_size, mode='bilinear').to(device)
                        output = model(images)
                        existing_results = model_outputs.get(model_name, [])
                        existing_results.append(output)
                        model_outputs[model_name] = existing_results

            true_labels.extend(label.item())

    for name, results in model_outputs.values():
        formatted_results = torch.cat(results, dim=0) #removing the batchsize of 1 by flattening it
        model_outputs[name] = formatted_results

    return model_outputs, true_labels



def train_svm(model_list:list, train_dataloader, test_dataloader=None, device:str='cuda', save_model:bool=False):
    """
    Function to train a svm
    Model list can either be nested list of length 1: [[[model, model_name, img_size], [model, model_name, img_size], ...]]
    - this tells the function that any pair of models can be chosen in this list

    Model list can also be nested with length 2: [[[model, model_name, img_size], [model, model_name, img_size]], [[model, model_name, img_size], [model, model_name, img_size]]]
    - this tells the function that each pair must contain 1 model from each list, no picking from the same list

    Args:
        model_list (list): Either a nested list of length 1 or 2
        train_dataloader: train dataloader to run outputs for
        test_dataloader (optional): test dataloader to run evaluation on, if None no evaluation will be done. Defaults to None.
        device (str, optional): cuda or cpu. Defaults to 'cuda'.
        save_model (bool, optional): boolean to save every svm model. Defaults to False.

    Raises:
        Exception: if model_list length is not 1 or 2

    Returns:
        recall_result (pd.DataFrame) : dataframe containing recall results in the format of [model1name, model2name, recallscore]
    """
    if len(model_list) != 1 or len(model_list) != 2:
        raise Exception("Currently only accepting nested list of length 1 or 2")
    
    recall_result = pd.DataFrame(columns=["Model1", "Model2", "Recall"])

    #generating model outputs (dataset for svm)
    print(f"---- Generating training data ----")
    model_output, true_labels = generate_model_outputs(model_list, train_dataloader, device)
    print(f"---- Generating testing data ----")
    if test_dataloader != None: 
        test_model_output, test_true_labels = generate_model_outputs(model_list, test_dataloader, device)

    #training svm based on combination
    if len(model_list) == 1:
        #can combine with ANY model in the list
        for i in range(len(model_list[0])):
            for j in range(i + 1, len(model_list[0])): 
                model_name_1 = model_list[0][i][1]
                model_name_2 = model_list[0][j][1]

                #concat model outputs to form train data
                train_data = torch.cat((model_output[model_name_1], model_output[model_name_2]), dim=1).cpu().detach().numpy()
                train_labels = np.array(true_labels)
                
                #initialise and train model
                svm_classifier = SVC(decision_function_shape='ovo')
                svm_classifier.fit(train_data, train_labels)

                #save model
                if save_model:
                    with open(f'../results/svm/{model_name_1}_{model_name_2}_svm.pkl', 'wb') as f:
                        pickle.dump(f)

                #concat test model outputs to form test data
                test_data = torch.cat((test_model_output[model_name_1], test_model_output[model_name_2]), dim=1).cpu().detach().numpy()
                test_labels = np.array(test_true_labels)

                #evaluate recall
                predictions = svm_classifier.predict(test_data)
                recall = recall_score(test_labels, predictions, average='weighted')

                print(f"{model_name_1}, {model_name_2} recall: {recall}")
                recall_result.loc[len(recall_result)] = [model_name_1, model_name_2, recall]


    elif len(model_list) == 2:
        #models in list 1 can only be paired with models in list 2
        for i in range(len(model_list[0])):
            for j in range(len(model_list[1])):
                model_name_1 = model_list[0][i][1]
                model_name_2 = model_list[0][j][1]

                #concat model outputs to form train data
                train_data = torch.cat((model_output[model_name_1], model_output[model_name_2]), dim=1).cpu().detach().numpy()
                train_labels = np.array(true_labels)
                
                #initialise and train model
                svm_classifier = SVC(decision_function_shape='ovo')
                svm_classifier.fit(train_data, train_labels)

                #save model
                if save_model:
                    with open(f'../results/svm/{model_name_1}_{model_name_2}_svm.pkl', 'wb') as f:
                        pickle.dump(f)

                #concat test model outputs to form test data
                test_data = torch.cat((test_model_output[model_name_1], test_model_output[model_name_2]), dim=1).cpu().detach().numpy()
                test_labels = np.array(test_true_labels)

                #evaluate recall
                predictions = svm_classifier.predict(test_data)
                recall = recall_score(test_labels, predictions, average='weighted')

                print(f"{model_name_1}, {model_name_2} recall: {recall}")
                recall_result.loc[len(recall_result)] = [model_name_1, model_name_2, recall]

    return recall_result

### Model

In [ ]:
class PreTrainedCNNModels(torch.nn.Module):
    def __init__(self, model_type:str, num_first_unfreeze:int, num_last_unfreeze:int, num_class:int):
        super(PreTrainedCNNModels, self).__init__()
        """
        Class that contains InceptionV3, Resnet50, Resnet152, EfficientNet, DenseNet, VGG16, MaxVit fine tuned models

        Args:
            model_type (str): Determines which pre-trained models to use
                              Must be: InceptionV3, Resnet50, Resnet152, EfficientNet, DenseNet, VGG16, MaxVit
            num_first_unfreeze (int): Number of first layers to unfreeze and finetune
            num_last_unfreeze (int): Number of layers to unfreeze and finetune
            num_class (int): Number of output classes for the classification
        """
        #selecting model type
        if model_type == 'InceptionV3':
            self.model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
            self.model.aux_logits = False

        elif model_type == 'Resnet50':
            self.model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

        elif model_type == 'Resnet152':
            self.model = models.resnet152(weights=models.ResNet152_Weights.DEFAULT)

        elif model_type == 'EfficientNet':
            self.model = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.DEFAULT)

        elif model_type == 'DenseNet':
            self.model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
        
        elif model_type == 'VGG16':
            self.model = models.vgg16_bn(weights=models.VGG16_BN_Weights.DEFAULT)

        elif model_type == 'MaxVit':
            self.model = models.maxvit_t(weights=models.MaxVit_T_Weights.DEFAULT)
        
        else:
            raise Exception("Invalid model type chosen. Please select one of the following\n[InceptionV3, Resnet50, Resnet152, EfficientNet, DenseNet, VGG16, MaxVit]")

        
        #modifying final layer
        if model_type in ['InceptionV3', 'Resnet50', 'Resnet152']:
            self.model.fc = torch.nn.Linear(self.model.fc.in_features, num_class)

        elif model_type == 'DenseNet':
            self.model.classifier = torch.nn.Linear(self.model.classifier.in_features, num_class)

        else:
            self.model.classifier[-1] = torch.nn.Linear(self.model.classifier[-1].in_features, num_class)


        model_paramteres = list(self.model.parameters())
        #unfreeze last num_last_unfreeze layers
        for param in model_paramteres[-num_last_unfreeze:]:
            param.requires_grad = True

        #unfreeze first num_first_unfreeze layers 
        for param in model_paramteres[:num_first_unfreeze]:
            param.requires_grad = True
            
        #freeze rest of the layers
        for param in model_paramteres[num_first_unfreeze:-num_last_unfreeze]:
            param.requires_grad = False


    def forward(self, images):
        return self.model(images)

### Training all model with equal class weights with unfrozen last linear layer

Training all model with Cross Entropy Loss without any class weights on Ben Graham dataset

Note: We have to put 2 as num_last_unfreeze due to the fact that the lineaer layer has bias and weights and is thus considered as 2 layers in the architecture.

In [ ]:
# Training 
models_list = [['InceptionV3', (299,299)],
               ['Resnet50', (224,224)],
               ['Resnet152',(224,224)],
               ['EfficientNet',(224,224)],
               ['DenseNet',(224,224)],
               ['VGG16',(224,224)],
               ['MaxVit', (224,224)]]


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 50

all_results = pd.DataFrame(columns=["Model", "train loss", "val loss", "val accuracy", "val sensitivity"])

for model_name, img_size in models_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    #reinitialise model, loss and optimizer
    model = PreTrainedCNNModels(model_name, 0, 2, len(ben_train_dataset.classes)).to(device)
    criterion=torch.nn.CrossEntropyLoss() #equal class weights
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

    print(f"-------------Training {model_name}------------")
    model_result = train(model,
                         criterion,
                         optimizer,
                         img_size,
                         ben_train_dataloader,
                         ben_val_dataloader,
                         'sensitivity',
                         num_epochs,
                         device,
                         f"{model_name}_0_2_base")
    
    del model #clear cuda memory
    
    all_results =  pd.concat([all_results, model_result])

all_results

In [ ]:
# Inference
models_list = [['InceptionV3', (299,299)],
               ['Resnet50', (224,224)],
               ['Resnet152',(224,224)],
               ['EfficientNet',(224,224)],
               ['DenseNet',(224,224)],
               ['VGG16',(224,224)],
               ['MaxVit', (224,224)]]


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for model_name, img_size in models_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    best_model = torch.load(f'../models/cnn/{model_name}_0_2_base_best.pt')
    last_model = torch.load(f'../model/cnn/{model_name}_0_2_base_last.pt')
    criterion=torch.nn.CrossEntropyLoss()

    print(f"-------------Evaluating {model_name}------------")
    #evaluating the best model
    loss, accuracy, sensitivity = eval(best_model,
                                        criterion,
                                        img_size,
                                        ben_test_dataloader,
                                        device)
    print(f"Best Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {sensitivity}")
    #evaluating the last model    
    loss, accuracy, sensitivity = eval(last_model,
                                        criterion,
                                        img_size,
                                        ben_test_dataloader,
                                        device)
    print(f"Last Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {sensitivity}\n")

### Training all model with calculated weighted class with unfrozen last linear layer

Training all model with Cross Entropy Loss with calculated class weights on Ben Graham dataset

Class Weights of i = Total Num of Samples / Total Num of samples of Class i * Num of classes

We calculate the average of samples per class if it was equally distributed and then calculate the class weights based on the difference of actual vs ideal

<br/>
Note: We have to put 2 as num_last_unfreeze due to the fact that the lineaer layer has bias and weights and is thus considered as 2 layers in the architecture.

In [ ]:
class_weights = calculate_dr_class_weights(ben_train_dataset)

In [ ]:
# Training 
models_list = [['InceptionV3', (299,299)],
               ['Resnet50', (224,224)],
               ['Resnet152',(224,224)],
               ['EfficientNet',(224,224)],
               ['DenseNet',(224,224)],
               ['VGG16',(224,224)],
               ['MaxVit', (224,224)]]


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 50

all_results = pd.DataFrame(columns=["Model", "train loss", "val loss", "val accuracy", "val sensitivity"])

for model_name, img_size in models_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    #reinitialise model, loss and optimizer
    model = PreTrainedCNNModels(model_name, 0, 2, len(ben_train_dataset.classes)).to(device)
    criterion=torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float32).to(device)) #calculated weights
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

    print(f"-------------Training {model_name}------------")
    model_result = train(model,
                         criterion,
                         optimizer,
                         img_size,
                         ben_train_dataloader,
                         ben_val_dataloader,
                         'sensitivity',
                         num_epochs,
                         device,
                         f"{model_name}_0_2_weighted")
    
    del model #clear cuda memory
    
    all_results =  pd.concat([all_results, model_result])

all_results

In [ ]:
#Inference
models_list = [['InceptionV3', (299,299)],
               ['Resnet50', (224,224)],
               ['Resnet152',(224,224)],
               ['EfficientNet',(224,224)],
               ['DenseNet',(224,224)],
               ['VGG16',(224,224)],
               ['MaxVit', (224,224)]]


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for model_name, img_size in models_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    best_model = torch.load(f'../models/cnn/{model_name}_0_2_weighted_best.pt')
    last_model = torch.load(f'../models/cnn/{model_name}_0_2_weighted_last.pt')
    criterion=torch.nn.CrossEntropyLoss()

    print(f"-------------Evaluating {model_name}------------")
    #evaluating the best model
    loss, accuracy, sensitivity = eval(best_model,
                                        criterion,
                                        img_size,
                                        ben_test_dataloader,
                                        device)
    print(f"Best Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {sensitivity}")
    #evaluating the last model    
    loss, accuracy, sensitivity = eval(last_model,
                                        criterion,
                                        img_size,
                                        ben_test_dataloader,
                                        device)
    print(f"Last Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {sensitivity}\n")

### Training all model with custom weighted class with unfrozen last linear layer

Training all model with Cross Entropy Loss with custom class weights [1, 6, 1, 10, 3] on Ben Graham dataset


Note: We have to put 2 as num_last_unfreeze due to the fact that the lineaer layer has bias and weights and is thus considered as 2 layers in the architecture.

In [ ]:
# Training 
models_list = [['InceptionV3', (299,299)],
               ['Resnet50', (224,224)],
               ['Resnet152',(224,224)],
               ['EfficientNet',(224,224)],
               ['DenseNet',(224,224)],
               ['VGG16',(224,224)],
               ['MaxVit', (224,224)]]


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 50

all_results = pd.DataFrame(columns=["Model", "train loss", "val loss", "val accuracy", "val sensitivity"])

for model_name, img_size in models_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    #reinitialise model, loss and optimizer
    model = PreTrainedCNNModels(model_name, 0, 2, len(ben_train_dataset.classes)).to(device)
    criterion=torch.nn.CrossEntropyLoss(weight=torch.tensor([1, 6, 1, 10, 3], dtype=torch.float32).to(device)) #custom weights
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

    print(f"-------------Training {model_name}------------")
    model_result = train(model,
                         criterion,
                         optimizer,
                         img_size,
                         ben_train_dataloader,
                         ben_val_dataloader,
                         'sensitivity',
                         num_epochs,
                         device,
                         f"{model_name}_0_2_custom_weighted")
    
    del model #clear cuda memory
    
    all_results =  pd.concat([all_results, model_result])
    
all_results

In [ ]:
models_list = [['InceptionV3', (299,299)],
               ['Resnet50', (224,224)],
               ['Resnet152',(224,224)],
               ['EfficientNet',(224,224)],
               ['DenseNet',(224,224)],
               ['VGG16',(224,224)],
               ['MaxVit', (224,224)]]


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for model_name, img_size in models_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    best_model = torch.load(f'../models/cnn/{model_name}_0_2_custom_weighted_best.pt')
    last_model = torch.load(f'../models/cnn/{model_name}_0_2_custom_weighted_last.pt')
    criterion=torch.nn.CrossEntropyLoss()

    print(f"-------------Evaluating {model_name}------------")
    #evaluating the best model
    loss, accuracy, sensitivity = eval(best_model,
                                        criterion,
                                        img_size,
                                        ben_test_dataloader,
                                        device)
    print(f"Best Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {sensitivity}")
    #evaluating the last model    
    loss, accuracy, sensitivity = eval(last_model,
                                        criterion,
                                        img_size,
                                        ben_test_dataloader,
                                        device)
    print(f"Last Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {sensitivity}\n")

### Training all model with calculated weighted class with 8 last unfrozen layers before training with SVM

Training all model with Cross Entropy Loss with calculated class weights on Ben Graham dataset with last 8 children layers unfreezed

This helps us decide which model shall we be focusing on for layer exploration

#### CNN Classification training

In [ ]:
class_weights = calculate_dr_class_weights(ben_train_dataset)

In [ ]:
#Training
models_list = [['InceptionV3', (299,299)],
               ['Resnet50', (224,224)],
               ['Resnet152',(224,224)],
               ['EfficientNet',(224,224)],
               ['DenseNet',(224,224)],
               ['VGG16',(224,224)],
               ['MaxVit', (224,224)]]


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 30

all_results = pd.DataFrame(columns=["Model", "train loss", "val loss", "val accuracy", "val sensitivity"])

for model_name, img_size in models_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    #reinitialise model, loss and optimizer
    model = PreTrainedCNNModels(model_name, 0, 8, len(ben_train_dataset.classes)).to(device)
    criterion=torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float32).to(device))
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

    print(f"-------------Training {model_name}------------")
    model_result = train(model,
                         criterion,
                         optimizer,
                         img_size,
                         ben_train_dataloader,
                         ben_val_dataloader,
                         'sensitivity',
                         num_epochs,
                         device,
                         f"{model_name}_0_8_weighted")
    
    del model #clear cuda memory

    all_results =  pd.concat([all_results, model_result])


all_results
all_results.to_csv("../results/cnn_8_weighted.csv")


In [ ]:
#Inference
models_list = [['InceptionV3', (299,299)],
               ['Resnet50', (224,224)],
               ['Resnet152',(224,224)],
               ['EfficientNet',(224,224)],
               ['DenseNet',(224,224)],
               ['VGG16',(224,224)],
               ['MaxVit', (224,224)]]

#Creating SVM model list
svm_model_list = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for model_name, img_size in models_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    best_model = torch.load(f'../models/cnn/{model_name}_0_8_weighted_best.pt')
    last_model = torch.load(f'../models/cnn/{model_name}_0_8_weighted_last.pt')
    criterion=torch.nn.CrossEntropyLoss()

    print(f"-------------Evaluating {model_name}------------")
    #evaluating the best model
    loss, accuracy, best_sensitivity = eval(best_model,
                                        criterion,
                                        img_size,
                                        ben_test_dataloader,
                                        device)
    print(f"Best Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {best_sensitivity}")
    #evaluating the last model    
    loss, accuracy, last_sensitivity = eval(last_model,
                                        criterion,
                                        img_size,
                                        ben_test_dataloader,
                                        device)
    print(f"Last Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {last_sensitivity}\n")

    #adding best model to svm_model_list
    if best_sensitivity > last_sensitivity:
        svm_model_list.append([best_model, f"{model_name}_0_8_weighted", img_size])
    
    else:
        svm_model_list.append([last_model, f"{model_name}_0_8_weighted", img_size])


#### SVM training

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

recall_results = train_svm([svm_model_list], ben_train_dataloader, ben_test_dataloader, device, True)

recall_results.to_csv("../results/svm_0_8_weighted.csv")

### DenseNet and MaxVit Layer Exploration with SVM for Ben Graham Dataset

Exploring different numbers of unfrozen layers in DenseNet and MaxViT with Ben Graham dataset before passing it through an SVM

The layers explored will be
- first 0 layer unfrozen and last layer unfrozen (done earlier)
- first 1 layer unfrozen and last layers unfrozen
- first 0 layer unfrozen and last 8 layers unfrozen (done earlier)
- first 1 layer unfrozen and last 8 layers unfrozen
- first 0 layer unfrozen and last 16 layers unfrozen
- first 1 layer unfrozen and last 16 layers unfrozen
- first 0 layer unfrozen and last 32 layers unfrozen
- first 0 layer unfrozen and last 32 layers unfrozen

#### DenseNet Layer Training

In [ ]:
class_weights = calculate_dr_class_weights(ben_train_dataset)

In [ ]:
# Training 
layer_list = [[1, 2], #ignoring [0,2] amd [0,8] as they are trained already
              [1, 8],
              [0, 16],
              [1, 16],
              [0, 32],
              [1, 32]]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 30

all_results = pd.DataFrame(columns=["Model", "train loss", "val loss", "val accuracy", "val sensitivity"])

for num_first_unfreeze, num_last_unfreeze in layer_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    #reinitialise model with correct weights, loss and optimizer
    model = PreTrainedCNNModels("DenseNet", num_first_unfreeze, num_last_unfreeze, len(ben_train_dataset.classes)).to(device)
    criterion=torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float32).to(device))
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

    print(f"-------------Training DenseNet with {num_first_unfreeze} Unfrozen First Layers and {num_last_unfreeze} Unfrozen Last Layers------------")
    model_result = train(model,
                         criterion,
                         optimizer,
                         img_size,
                         ben_train_dataloader,
                         ben_val_dataloader,
                         'sensitivity',
                         num_epochs,
                         device,
                         f"DenseNet_{num_first_unfreeze}_{num_last_unfreeze}_weighted")
    
    all_results =  pd.concat([all_results, model_result])

all_results

In [ ]:
# Inference
layer_list = [[0, 2],
              [1, 2],
              [0, 8],
              [1, 8],
              [0, 16],
              [1, 16],
              [0, 32],
              [1, 32]]

#Creating densenet SVM model list
densenet_svm_model_list = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for num_first_unfreeze, num_last_unfreeze in layer_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    best_model = torch.load(f'../models/cnn/DenseNet_{num_first_unfreeze}_{num_last_unfreeze}_weighted_best.pt')
    last_model = torch.load(f'../models/cnn/DenseNet_{num_first_unfreeze}_{num_last_unfreeze}_weighted_last.pt')
    criterion=torch.nn.CrossEntropyLoss()

    print(f"-------------Evaluating DenseNet with {num_first_unfreeze} Unfrozen First Layers and {num_last_unfreeze} Unfrozen Last Layers------------")
    #evaluating the best model
    loss, accuracy, best_sensitivity = eval(best_model,
                                        criterion,
                                        img_size,
                                        ben_test_dataloader,
                                        device)
    print(f"Best Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {best_sensitivity}")
    #evaluating the last model    
    loss, accuracy, last_sensitivity = eval(last_model,
                                        criterion,
                                        img_size,
                                        ben_test_dataloader,
                                        device)
    print(f"Last Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {last_sensitivity}\n")

    #adding best model to svm_model_list
    if best_sensitivity > last_sensitivity:
        densenet_svm_model_list.append([best_model, f"DenseNet_{num_first_unfreeze}_{num_last_unfreeze}_weighted", (224,224)])
    
    else:
        densenet_svm_model_list.append([last_model, f"DenseNet_{num_first_unfreeze}_{num_last_unfreeze}_weighted", (224,224)])


#### MaxVit Layer Training

In [ ]:
class_weights = calculate_dr_class_weights(ben_train_dataset)

In [ ]:
# Training 
layer_list = [[1, 2], #ignoring [0,2] amd [0,8] as they are trained already
              [1, 8],
              [0, 16],
              [1, 16],
              [0, 32],
              [1, 32]]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 30

all_results = pd.DataFrame(columns=["Model", "train loss", "val loss", "val accuracy", "val sensitivity"])

for num_first_unfreeze, num_last_unfreeze in layer_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    #reinitialise model with correct weights, loss and optimizer
    model = PreTrainedCNNModels("MaxVit", num_first_unfreeze, num_last_unfreeze, len(ben_train_dataset.classes)).to(device)
    criterion=torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float32).to(device))
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

    print(f"-------------Training MaxVit with {num_first_unfreeze} Unfrozen First Layers and {num_last_unfreeze} Unfrozen Last Layers------------")
    model_result = train(model,
                         criterion,
                         optimizer,
                         img_size,
                         ben_train_dataloader,
                         ben_val_dataloader,
                         'sensitivity',
                         num_epochs,
                         device,
                         f"MaxVit_{num_first_unfreeze}_{num_last_unfreeze}_weighted")
    
    all_results =  pd.concat([all_results, model_result])

all_results

In [ ]:
# Inference
layer_list = [[0, 2],
              [1, 2],
              [0, 8],
              [1, 8],
              [0, 16],
              [1, 16],
              [0, 32],
              [1, 32]]

#Creating densenet SVM model list
maxvit_svm_model_list = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for num_first_unfreeze, num_last_unfreeze in layer_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    best_model = torch.load(f'../models/cnn/MaxVit_{num_first_unfreeze}_{num_last_unfreeze}_weighted_best.pt')
    last_model = torch.load(f'../models/cnn/MaxVit_{num_first_unfreeze}_{num_last_unfreeze}_weighted_last.pt')
    criterion=torch.nn.CrossEntropyLoss()

    print(f"-------------Evaluating MaxVit with {num_first_unfreeze} Unfrozen First Layers and {num_last_unfreeze} Unfrozen Last Layers------------")
    #evaluating the best model
    loss, accuracy, best_sensitivity = eval(best_model,
                                        criterion,
                                        img_size,
                                        ben_test_dataloader,
                                        device)
    print(f"Best Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {best_sensitivity}")
    #evaluating the last model    
    loss, accuracy, last_sensitivity = eval(last_model,
                                        criterion,
                                        img_size,
                                        ben_test_dataloader,
                                        device)
    print(f"Last Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {last_sensitivity}\n")

    #adding best model to svm_model_list
    if best_sensitivity > last_sensitivity:
        maxvit_svm_model_list.append([best_model, f"MaxVit_{num_first_unfreeze}_{num_last_unfreeze}_weighted", (224,224)])
    
    else:
        maxvit_svm_model_list.append([last_model, f"MaxVit_{num_first_unfreeze}_{num_last_unfreeze}_weighted", (224,224)])


#### SVM training

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

recall_results = train_svm([densenet_svm_model_list, maxvit_svm_model_list], ben_train_dataloader, ben_test_dataloader, device, True)

recall_results.to_csv("../results/densenet_maxvit_ben.csv")

### DenseNet and MaxVit Layer Exploration with SVM for OPENCV Dataset

Exploring different numbers of unfrozen layers in DenseNet and MaxViT with OPENCV dataset before passing it through an SVM

The layers explored will be
- first 0 layer unfrozen and last layer unfrozen (done earlier)
- first 1 layer unfrozen and last layers unfrozen
- first 0 layer unfrozen and last 8 layers unfrozen (done earlier)
- first 1 layer unfrozen and last 8 layers unfrozen
- first 0 layer unfrozen and last 16 layers unfrozen
- first 1 layer unfrozen and last 16 layers unfrozen
- first 0 layer unfrozen and last 32 layers unfrozen
- first 0 layer unfrozen and last 32 layers unfrozen

#### DenseNet Layer Training

In [ ]:
class_weights = calculate_dr_class_weights(opencv_train_dataset)

In [ ]:
# Training 
layer_list = [[0, 2],
              [1, 2],
              [0, 8],
              [1, 8],
              [0, 16],
              [1, 16],
              [0, 32],
              [1, 32]]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 30

all_results = pd.DataFrame(columns=["Model", "train loss", "val loss", "val accuracy", "val sensitivity"])

for num_first_unfreeze, num_last_unfreeze in layer_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    #reinitialise model with correct weights, loss and optimizer
    model = PreTrainedCNNModels("DenseNet", num_first_unfreeze, num_last_unfreeze, len(opencv_train_dataset.classes)).to(device)
    criterion=torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float32).to(device))
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

    print(f"-------------Training DenseNet with {num_first_unfreeze} Unfrozen First Layers and {num_last_unfreeze} Unfrozen Last Layers------------")
    model_result = train(model,
                         criterion,
                         optimizer,
                         img_size,
                         opencv_train_dataloader,
                         opencv_val_dataloader,
                         'sensitivity',
                         num_epochs,
                         device,
                         f"DenseNet_opencv_{num_first_unfreeze}_{num_last_unfreeze}_weighted")
    
    all_results =  pd.concat([all_results, model_result])

all_results

In [ ]:
# Inference
layer_list = [[0, 2],
              [1, 2],
              [0, 8],
              [1, 8],
              [0, 16],
              [1, 16],
              [0, 32],
              [1, 32]]

#Creating densenet SVM model list
densenet_svm_model_list = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for num_first_unfreeze, num_last_unfreeze in layer_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    best_model = torch.load(f'../models/cnn/DenseNet_opencv_{num_first_unfreeze}_{num_last_unfreeze}_weighted_best.pt')
    last_model = torch.load(f'../models/cnn/DenseNet_opencv_{num_first_unfreeze}_{num_last_unfreeze}_weighted_last.pt')
    criterion=torch.nn.CrossEntropyLoss()

    print(f"-------------Evaluating DenseNet with {num_first_unfreeze} Unfrozen First Layers and {num_last_unfreeze} Unfrozen Last Layers------------")
    #evaluating the best model
    loss, accuracy, best_sensitivity = eval(best_model,
                                        criterion,
                                        img_size,
                                        opencv_test_dataloader,
                                        device)
    print(f"Best Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {best_sensitivity}")
    #evaluating the last model    
    loss, accuracy, last_sensitivity = eval(last_model,
                                        criterion,
                                        img_size,
                                        opencv_test_dataloader,
                                        device)
    print(f"Last Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {last_sensitivity}\n")

    #adding best model to svm_model_list
    if best_sensitivity > last_sensitivity:
        densenet_svm_model_list.append([best_model, f"DenseNet_opencv_{num_first_unfreeze}_{num_last_unfreeze}_weighted", (224,224)])
    
    else:
        densenet_svm_model_list.append([last_model, f"DenseNet_opencv_{num_first_unfreeze}_{num_last_unfreeze}_weighted", (224,224)])


#### MaxVit Layer Training

In [ ]:
class_weights = calculate_dr_class_weights(opencv_train_dataset)

In [ ]:
# Training 
layer_list = [[0, 2],
              [1, 2],
              [0, 8],
              [1, 8],
              [0, 16],
              [1, 16],
              [0, 32],
              [1, 32]]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 30

all_results = pd.DataFrame(columns=["Model", "train loss", "val loss", "val accuracy", "val sensitivity"])

for num_first_unfreeze, num_last_unfreeze in layer_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    #reinitialise model with correct weights, loss and optimizer
    model = PreTrainedCNNModels("MaxVit", num_first_unfreeze, num_last_unfreeze, len(opencv_train_dataset.classes)).to(device)
    criterion=torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float32).to(device))
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

    print(f"-------------Training MaxVit with {num_first_unfreeze} Unfrozen First Layers and {num_last_unfreeze} Unfrozen Last Layers------------")
    model_result = train(model,
                         criterion,
                         optimizer,
                         img_size,
                         opencv_train_dataloader,
                         opencv_val_dataloader,
                         'sensitivity',
                         num_epochs,
                         device,
                         f"MaxVit_opencv_{num_first_unfreeze}_{num_last_unfreeze}_weighted")
    
    all_results =  pd.concat([all_results, model_result])

all_results

In [ ]:
# Inference
layer_list = [[0, 2],
              [1, 2],
              [0, 8],
              [1, 8],
              [0, 16],
              [1, 16],
              [0, 32],
              [1, 32]]

#Creating densenet SVM model list
maxvit_svm_model_list = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for num_first_unfreeze, num_last_unfreeze in layer_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    best_model = torch.load(f'../models/cnn/MaxVit_opencv_{num_first_unfreeze}_{num_last_unfreeze}_weighted_best.pt')
    last_model = torch.load(f'../models/cnn/MaxVit_opencv_{num_first_unfreeze}_{num_last_unfreeze}_weighted_last.pt')
    criterion=torch.nn.CrossEntropyLoss()

    print(f"-------------Evaluating MaxVit with {num_first_unfreeze} Unfrozen First Layers and {num_last_unfreeze} Unfrozen Last Layers------------")
    #evaluating the best model
    loss, accuracy, best_sensitivity = eval(best_model,
                                        criterion,
                                        img_size,
                                        opencv_test_dataloader,
                                        device)
    print(f"Best Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {best_sensitivity}")
    #evaluating the last model    
    loss, accuracy, last_sensitivity = eval(last_model,
                                        criterion,
                                        img_size,
                                        ben_test_dataloader,
                                        device)
    print(f"Last Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {last_sensitivity}\n")

    #adding best model to svm_model_list
    if best_sensitivity > last_sensitivity:
        maxvit_svm_model_list.append([best_model, f"MaxVit_opencv_{num_first_unfreeze}_{num_last_unfreeze}_weighted", (224,224)])
    
    else:
        maxvit_svm_model_list.append([last_model, f"MaxVit_opencv_{num_first_unfreeze}_{num_last_unfreeze}_weighted", (224,224)])


#### SVM training

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

recall_results = train_svm([densenet_svm_model_list, maxvit_svm_model_list], opencv_train_dataloader, opencv_test_dataloader, device, True)

recall_results.to_csv("../results/densenet_maxvit_opencv.csv")

### DenseNet and MaxVit Layer Exploration with SVM for UNET_Binary Dataset

Exploring different numbers of unfrozen layers in DenseNet and MaxViT with UNET_Binary dataset before passing it through an SVM

The layers explored will be
- first 0 layer unfrozen and last layer unfrozen (done earlier)
- first 1 layer unfrozen and last layers unfrozen
- first 0 layer unfrozen and last 8 layers unfrozen (done earlier)
- first 1 layer unfrozen and last 8 layers unfrozen
- first 0 layer unfrozen and last 16 layers unfrozen
- first 1 layer unfrozen and last 16 layers unfrozen
- first 0 layer unfrozen and last 32 layers unfrozen
- first 0 layer unfrozen and last 32 layers unfrozen

#### DenseNet Layer Training

In [ ]:
class_weights = calculate_dr_class_weights(unetb_train_dataset)

In [ ]:
# Training 
layer_list = [[0, 2],
              [1, 2],
              [0, 8],
              [1, 8],
              [0, 16],
              [1, 16],
              [0, 32],
              [1, 32]]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 30

all_results = pd.DataFrame(columns=["Model", "train loss", "val loss", "val accuracy", "val sensitivity"])

for num_first_unfreeze, num_last_unfreeze in layer_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    #reinitialise model with correct weights, loss and optimizer
    model = PreTrainedCNNModels("DenseNet", num_first_unfreeze, num_last_unfreeze, len(unetb_train_dataset.classes)).to(device)
    criterion=torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float32).to(device))
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

    print(f"-------------Training DenseNet with {num_first_unfreeze} Unfrozen First Layers and {num_last_unfreeze} Unfrozen Last Layers------------")
    model_result = train(model,
                         criterion,
                         optimizer,
                         img_size,
                         unetb_train_dataloader,
                         unetb_val_dataloader,
                         'sensitivity',
                         num_epochs,
                         device,
                         f"DenseNet_unetb_{num_first_unfreeze}_{num_last_unfreeze}_weighted")
    
    all_results =  pd.concat([all_results, model_result])

all_results

In [ ]:
# Inference
layer_list = [[0, 2],
              [1, 2],
              [0, 8],
              [1, 8],
              [0, 16],
              [1, 16],
              [0, 32],
              [1, 32]]

#Creating densenet SVM model list
densenet_svm_model_list = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for num_first_unfreeze, num_last_unfreeze in layer_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    best_model = torch.load(f'../models/cnn/DenseNet_unetb_{num_first_unfreeze}_{num_last_unfreeze}_weighted_best.pt')
    last_model = torch.load(f'../models/cnn/DenseNet_unetb_{num_first_unfreeze}_{num_last_unfreeze}_weighted_last.pt')
    criterion=torch.nn.CrossEntropyLoss()

    print(f"-------------Evaluating DenseNet with {num_first_unfreeze} Unfrozen First Layers and {num_last_unfreeze} Unfrozen Last Layers------------")
    #evaluating the best model
    loss, accuracy, best_sensitivity = eval(best_model,
                                        criterion,
                                        img_size,
                                        unetb_test_dataloader,
                                        device)
    print(f"Best Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {best_sensitivity}")
    #evaluating the last model    
    loss, accuracy, last_sensitivity = eval(last_model,
                                        criterion,
                                        img_size,
                                        unetb_test_dataloader,
                                        device)
    print(f"Last Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {last_sensitivity}\n")

    #adding best model to svm_model_list
    if best_sensitivity > last_sensitivity:
        densenet_svm_model_list.append([best_model, f"DenseNet_unetb_{num_first_unfreeze}_{num_last_unfreeze}_weighted", (224,224)])
    
    else:
        densenet_svm_model_list.append([last_model, f"DenseNet_unetb_{num_first_unfreeze}_{num_last_unfreeze}_weighted", (224,224)])


#### MaxVit Layer Training

In [ ]:
class_weights = calculate_dr_class_weights(unetb_train_dataset)

In [ ]:
# Training 
layer_list = [[0, 2],
              [1, 2],
              [0, 8],
              [1, 8],
              [0, 16],
              [1, 16],
              [0, 32],
              [1, 32]]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 30

all_results = pd.DataFrame(columns=["Model", "train loss", "val loss", "val accuracy", "val sensitivity"])

for num_first_unfreeze, num_last_unfreeze in layer_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    #reinitialise model with correct weights, loss and optimizer
    model = PreTrainedCNNModels("MaxVit", num_first_unfreeze, num_last_unfreeze, len(unetb_train_dataset.classes)).to(device)
    criterion=torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float32).to(device))
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

    print(f"-------------Training MaxVit with {num_first_unfreeze} Unfrozen First Layers and {num_last_unfreeze} Unfrozen Last Layers------------")
    model_result = train(model,
                         criterion,
                         optimizer,
                         img_size,
                         unetb_train_dataloader,
                         unetb_val_dataloader,
                         'sensitivity',
                         num_epochs,
                         device,
                         f"MaxVit_unetb_{num_first_unfreeze}_{num_last_unfreeze}_weighted")
    
    all_results =  pd.concat([all_results, model_result])

all_results

In [ ]:
# Inference
layer_list = [[0, 2],
              [1, 2],
              [0, 8],
              [1, 8],
              [0, 16],
              [1, 16],
              [0, 32],
              [1, 32]]

#Creating densenet SVM model list
maxvit_svm_model_list = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for num_first_unfreeze, num_last_unfreeze in layer_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    best_model = torch.load(f'../models/cnn/MaxVit_unetb_{num_first_unfreeze}_{num_last_unfreeze}_weighted_best.pt')
    last_model = torch.load(f'../models/cnn/MaxVit_unetb_{num_first_unfreeze}_{num_last_unfreeze}_weighted_last.pt')
    criterion=torch.nn.CrossEntropyLoss()

    print(f"-------------Evaluating DenseNet with {num_first_unfreeze} Unfrozen First Layers and {num_last_unfreeze} Unfrozen Last Layers------------")
    #evaluating the best model
    loss, accuracy, best_sensitivity = eval(best_model,
                                        criterion,
                                        img_size,
                                        unetb_test_dataloader,
                                        device)
    print(f"Best Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {best_sensitivity}")
    #evaluating the last model    
    loss, accuracy, last_sensitivity = eval(last_model,
                                        criterion,
                                        img_size,
                                        unetb_test_dataloader,
                                        device)
    print(f"Last Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {last_sensitivity}\n")

    #adding best model to svm_model_list
    if best_sensitivity > last_sensitivity:
        maxvit_svm_model_list.append([best_model, f"MaxVit_unetb_{num_first_unfreeze}_{num_last_unfreeze}_weighted", (224,224)])
    
    else:
        maxvit_svm_model_list.append([last_model, f"MaxVit_unetb_{num_first_unfreeze}_{num_last_unfreeze}_weighted", (224,224)])


#### SVM training

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

recall_results = train_svm([densenet_svm_model_list, maxvit_svm_model_list], unetb_train_dataloader, unetb_test_dataloader, device, True)

recall_results.to_csv("../results/densenet_maxvit_unetb.csv")

### DenseNet and MaxVit Layer Exploration with SVM for UNET_Multiclass Dataset

Exploring different numbers of unfrozen layers in DenseNet and MaxViT with UNET_Multiclass dataset before passing it through an SVM

The layers explored will be
- first 0 layer unfrozen and last layer unfrozen (done earlier)
- first 1 layer unfrozen and last layers unfrozen
- first 0 layer unfrozen and last 8 layers unfrozen (done earlier)
- first 1 layer unfrozen and last 8 layers unfrozen
- first 0 layer unfrozen and last 16 layers unfrozen
- first 1 layer unfrozen and last 16 layers unfrozen
- first 0 layer unfrozen and last 32 layers unfrozen
- first 0 layer unfrozen and last 32 layers unfrozen

#### DenseNet Layer Training

In [ ]:
class_weights = calculate_dr_class_weights(unetm_train_dataset)

In [ ]:
# Training 
layer_list = [[0, 2],
              [1, 2],
              [0, 8],
              [1, 8],
              [0, 16],
              [1, 16],
              [0, 32],
              [1, 32]]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 30

all_results = pd.DataFrame(columns=["Model", "train loss", "val loss", "val accuracy", "val sensitivity"])

for num_first_unfreeze, num_last_unfreeze in layer_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    #reinitialise model with correct weights, loss and optimizer
    model = PreTrainedCNNModels("DenseNet", num_first_unfreeze, num_last_unfreeze, len(unetm_train_dataset.classes)).to(device)
    criterion=torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float32).to(device))
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

    print(f"-------------Training DenseNet with {num_first_unfreeze} Unfrozen First Layers and {num_last_unfreeze} Unfrozen Last Layers------------")
    model_result = train(model,
                         criterion,
                         optimizer,
                         img_size,
                         unetm_train_dataloader,
                         unetm_val_dataloader,
                         'sensitivity',
                         num_epochs,
                         device,
                         f"DenseNet_unetm_{num_first_unfreeze}_{num_last_unfreeze}_weighted")
    
    all_results =  pd.concat([all_results, model_result])

all_results

In [ ]:
# Inference
layer_list = [[0, 2],
              [1, 2],
              [0, 8],
              [1, 8],
              [0, 16],
              [1, 16],
              [0, 32],
              [1, 32]]

#Creating densenet SVM model list
densenet_svm_model_list = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for num_first_unfreeze, num_last_unfreeze in layer_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    best_model = torch.load(f'../models/cnn/DenseNet_unetm_{num_first_unfreeze}_{num_last_unfreeze}_weighted_best.pt')
    last_model = torch.load(f'../models/cnn/DenseNet_unetm_{num_first_unfreeze}_{num_last_unfreeze}_weighted_last.pt')
    criterion=torch.nn.CrossEntropyLoss()

    print(f"-------------Evaluating DenseNet with {num_first_unfreeze} Unfrozen First Layers and {num_last_unfreeze} Unfrozen Last Layers------------")
    #evaluating the best model
    loss, accuracy, best_sensitivity = eval(best_model,
                                        criterion,
                                        img_size,
                                        unetm_test_dataloader,
                                        device)
    print(f"Best Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {best_sensitivity}")
    #evaluating the last model    
    loss, accuracy, last_sensitivity = eval(last_model,
                                        criterion,
                                        img_size,
                                        unetm_test_dataloader,
                                        device)
    print(f"Last Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {last_sensitivity}\n")

    #adding best model to svm_model_list
    if best_sensitivity > last_sensitivity:
        densenet_svm_model_list.append([best_model, f"DenseNet_unetm_{num_first_unfreeze}_{num_last_unfreeze}_weighted", (224,224)])
    
    else:
        densenet_svm_model_list.append([last_model, f"DenseNet_unetm_{num_first_unfreeze}_{num_last_unfreeze}_weighted", (224,224)])


#### MaxVit Layer Training

In [ ]:
class_weights = calculate_dr_class_weights(unetm_train_dataset)

In [ ]:
# Training 
layer_list = [[0, 2],
              [1, 2],
              [0, 8],
              [1, 8],
              [0, 16],
              [1, 16],
              [0, 32],
              [1, 32]]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 30

all_results = pd.DataFrame(columns=["Model", "train loss", "val loss", "val accuracy", "val sensitivity"])

for num_first_unfreeze, num_last_unfreeze in layer_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    #reinitialise model with correct weights, loss and optimizer
    model = PreTrainedCNNModels("MaxVit", num_first_unfreeze, num_last_unfreeze, len(unetm_train_dataset.classes)).to(device)
    criterion=torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float32).to(device))
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

    print(f"-------------Training MaxVit with {num_first_unfreeze} Unfrozen First Layers and {num_last_unfreeze} Unfrozen Last Layers------------")
    model_result = train(model,
                         criterion,
                         optimizer,
                         img_size,
                         unetm_train_dataloader,
                         unetm_val_dataloader,
                         'sensitivity',
                         num_epochs,
                         device,
                         f"MaxVit_unetm_{num_first_unfreeze}_{num_last_unfreeze}_weighted")
    
    all_results =  pd.concat([all_results, model_result])

all_results

In [ ]:
# Inference
layer_list = [[0, 2],
              [1, 2],
              [0, 8],
              [1, 8],
              [0, 16],
              [1, 16],
              [0, 32],
              [1, 32]]

#Creating densenet SVM model list
maxvit_svm_model_list = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for num_first_unfreeze, num_last_unfreeze in layer_list:

    #clear cuda memory
    torch.cuda.empty_cache()
    gc.collect()

    best_model = torch.load(f'../models/cnn/MaxVit_unetm_{num_first_unfreeze}_{num_last_unfreeze}_weighted_best.pt')
    last_model = torch.load(f'../models/cnn/MaxVit_unetm_{num_first_unfreeze}_{num_last_unfreeze}_weighted_last.pt')
    criterion=torch.nn.CrossEntropyLoss()

    print(f"-------------Evaluating DenseNet with {num_first_unfreeze} Unfrozen First Layers and {num_last_unfreeze} Unfrozen Last Layers------------")
    #evaluating the best model
    loss, accuracy, best_sensitivity = eval(best_model,
                                        criterion,
                                        img_size,
                                        unetm_test_dataloader,
                                        device)
    print(f"Best Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {best_sensitivity}")
    #evaluating the last model    
    loss, accuracy, last_sensitivity = eval(last_model,
                                        criterion,
                                        img_size,
                                        unetm_test_dataloader,
                                        device)
    print(f"Last Model - Test Loss: {loss}, Test Accuracy: {accuracy}, Test Sensitivity: {last_sensitivity}\n")

    #adding best model to svm_model_list
    if best_sensitivity > last_sensitivity:
        maxvit_svm_model_list.append([best_model, f"MaxVit_unetm_{num_first_unfreeze}_{num_last_unfreeze}_weighted", (224,224)])
    
    else:
        maxvit_svm_model_list.append([last_model, f"MaxVit_unetm_{num_first_unfreeze}_{num_last_unfreeze}_weighted", (224,224)])


#### SVM training

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

recall_results = train_svm([densenet_svm_model_list, maxvit_svm_model_list], unetm_train_dataloader, unetm_test_dataloader, device, True)

recall_results.to_csv("../results/densenet_maxvit_unetm.csv")